<a href="https://colab.research.google.com/github/armandoordonez/eda_couse/blob/main/3_9_pipeline_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [ ]:
# Based on https://towardsdatascience.com/pipelines-custom-transformers-in-scikit-learn-the-step-by-step-guide-with-python-code-4a7d9b068156


# importamos  las librerias

import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

#Pipeline Experiment

In [ ]:
# Creamos un dataframe de prueba para analizar los cambios
### y = X1 + 2 * sqrt(X2)
# Esto asegura que un modelo de regresión lineal simple no pueda ajustarse perfectamente.

df = pd.DataFrame(columns=['X1', 'X2', 'y'], data=[
                                                   [1,16,9],
                                                   [4,36,16],
                                                   [1,16,9],
                                                   [2,9,8],
                                                   [3,36,15],
                                                   [2,49,16],
                                                   [4,25,14],
                                                   [5,36,17]
])


In [ ]:
df

,X1,X2,y
0,1,16,9
1,4,36,16
2,1,16,9
3,2,9,8
4,3,36,15
5,2,49,16
6,4,25,14
7,5,36,17


In [ ]:
# Separamos en train y test

train = df.iloc[:6]
test = df.iloc[6:]

train_X = train.drop('y', axis=1)
train_y = train.y

test_X = test.drop('y', axis=1)
test_y = test.y
train_X

,X1,X2
0,1,16
1,4,36
2,1,16
3,2,9
4,3,36
5,2,49


In [ ]:
test_X

,X1,X2
6,4,25
7,5,36


In [ ]:
#  verificacmos si la regresión lineal puede predecir correctamente

m1 = LinearRegression()
fit1 = m1.fit(train_X, train_y)
preds = fit1.predict(test_X)
print(f"\n{preds}")
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds))}\n")



[13.72113586 16.93334467]
RMSE: 0.20274138822160603



In [ ]:
# Las predicciones no son malas, pero hagamos algunos cálculos sobre las características de entrada para mejorar
# ¿Qué pasa si sacamos la raíz cuadrada de X2 y multiplicamos por 2?

train_X.X2 = 2 * np.sqrt(train_X.X2)
test_X.X2 = 2 * np.sqrt(test_X.X2)

print(test_X)

m2 = LinearRegression()

fit2 = m2.fit(train_X, train_y)

preds = fit2.predict(test_X)

print(f"\n{preds}")

print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds))}\n")


   X1    X2
6   4  10.0
7   5  12.0

[14. 17.]
RMSE: 0.0



In [ ]:
# una predicción perfecta, porque los datos después de la transformación se ajustan a una tendencia lineal perfecta.

In [ ]:
# Restauremos los datos a sus valores originales y hagámoslo mediante transformadores personalizados mediante canalización.

train = df.iloc[:6]
test = df.iloc[6:]

train_X = train.drop('y', axis=1)
train_y = train.y

test_X = test.drop('y', axis=1)
test_y = test.y

In [ ]:
# Creamos una clase transformadora

#__init__: este es el constructor. Se llama cuando se inicializa el pipeline.

#fit(): Se llama cuando hacemos fit en el pipeline.

#transform(): se llama cuando usamos fit o transform en el pipeline

class ExperimentalTransformer(BaseEstimator, TransformerMixin):

  def __init__(self):
    print('\n>>>>>>>init() called.\n')

  def fit(self, X, y = None):
    print('\n>>>>>>>fit() called.\n')
    return self

  def transform(self, X, y = None):
    print('\n>>>>>>>transform() called.\n')
    X_ = X.copy() # creamos una copia para evitar cambios en el conjunto de datos original
    X_.X2 = 2 * np.sqrt(X_.X2)
    return X_

In [ ]:
# sin transformación de entrada - para validar que obtenemos los mismos resultados que antes


print("create pipeline 1")
pipe1 = Pipeline(steps=[
                       ('linear_model', LinearRegression())
])

print("fit pipeline 1")
pipe1.fit(train_X, train_y)

print("predict via pipeline 1")
preds1 = pipe1.predict(test_X)

print(f"\n{preds1}")  # should be [13.72113586 16.93334467] RMSE: 0.20274138822160603
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds1))}\n")

create pipeline 1
fit pipeline 1
predict via pipeline 1

[13.72113586 16.93334467]
RMSE: 0.20274138822160603



In [ ]:
# Ahora aplicamos la transformación

print("create pipeline 2")
pipe2 = Pipeline(steps=[
                       ('experimental_trans', ExperimentalTransformer()),    # esto dispara una llamda a __init__
                       ('linear_model', LinearRegression())
])

# una sintaxis alternativa y más corta para hacer lo anterior, sin nombrar cada paso, es:
#pipe2 = make_pipeline(ExperimentalTransformer(), LinearRegression())

print("fit pipeline 2")
pipe2.fit(train_X, train_y)

print("predict via pipeline 2")
preds2 = pipe2.predict(test_X)

print(f"\n{preds2}")  # should be [14. 17.]
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds2))}\n")


# a. __init__ se llama cuando inicializamos pipe2.

# b. fit() y transform() se llaman cuando llamamos fit() con los datos de entrenamiento.
# Esto es necesario para transformar las características de entrada mientras se intenta predecir train_y.

# c. transform() se llama  cuando usamos predict(test_X): las características de prueba de entrada deben tener raíz cuadrada y duplicarse también antes de hacer predicciones.

create pipeline 2

>>>>>>>init() called.

fit pipeline 2

>>>>>>>fit() called.


>>>>>>>transform() called.

predict via pipeline 2

>>>>>>>transform() called.


[14. 17.]
RMSE: 0.0



In [ ]:
# Hemos asumido en la función transform() de nuestro ExperimentalTransformer que el nombre de la columna es X2. No lo hagamos y
# pasar el nombre de la columna a través del constructor, __init__()

In [ ]:
class ExperimentalTransformer_2(BaseEstimator, TransformerMixin):

  # agregue otro parámetro adicional, solo para mostrar su uso, mientras estamos en ello

  def __init__(self, feature_name, additional_param = "Pandebono"):
    print('\n>>>>>>>init() ExperimentalTransformer_2 called.\n')
    self.feature_name = feature_name
    self.additional_param = additional_param

  def fit(self, X, y = None):
    print('\n>>>>>>>fit() called.\n')
    print(f'\n additional param ExperimentalTransformer_2 ~~~~~ {self.additional_param}\n')
    return self

  def transform(self, X, y = None):
    print('\n>>>>>>>transform() called.\n')
    X_ = X.copy() # creating a copy to avoid changes to original dataset
    X_[self.feature_name] = 2 * np.sqrt(X_[self.feature_name])
    return X_

In [ ]:
# tenga cuidado de mantener el nombre del parámetro exactamente igual en el argumento de la función, así como
# la variable de la clase (feature_name). Cambiar eso causará problemas más adelante cuando también
# intenta transformar la característica objetivo (y). Provoca una doble llamada a __init__ por algún motivo.

In [ ]:
print("create pipeline 2")
pipe2 = Pipeline(steps=[
                       ('experimental_trans', ExperimentalTransformer_2('X2',additional_param='Parametro adicional')),
                       ('linear_model', LinearRegression())
])
print("fit pipeline 2")
pipe2.fit(train_X, train_y)
print("predict via pipeline 2")
preds2 = pipe2.predict(test_X)
print(f"\n{preds2}")  # should be [14. 17.]
print(f"RMSE: {np.sqrt(mean_squared_error(test_y, preds2))}\n")

create pipeline 2

>>>>>>>init() ExperimentalTransformer_2 called.

fit pipeline 2

>>>>>>>fit() called.


 additional param ExperimentalTransformer_2 ~~~~~ Parametro adicional


>>>>>>>transform() called.

predict via pipeline 2

>>>>>>>transform() called.


[14. 17.]
RMSE: 0.0

